In [1]:
from IPython.core.display import display, HTML 
display(HTML("<style>.container { width:100% !important; }</style>"))

In [17]:
import importlib # использую для перегрузки библиотек
import time
from IPython.display import clear_output
import random

import pandas as pd
import json
import numpy as np
import artm

import data_preparing
import artm_model
import work_with_model as wwm

importlib.reload(data_preparing)
importlib.reload(artm_model)
importlib.reload(wwm)
clear_output(True)

artm.version() 0.10.0


In [7]:
inf = {}
inf['exp_name'] = 'simple'
inf['test_size'] = 0.1
inf['num_collection_passes'] = 1
inf['topic_number'] = 750
inf['data_path'] = "service_cards_tokenised_remont_only"

In [12]:
inf['docs'] = data_preparing.read_and_prepared(inf['data_path'])
inf['path_train'], inf['path_test'] = data_preparing.form_test_train_set(docs=inf['docs'], name=inf['exp_name'], test_size=inf['test_size'])
inf['path_castom'] = data_preparing.form_castom_set(inf['docs'], name=inf['exp_name'])
inf['model'], inf['theta_train'] = artm_model.create_and_learn_PLSA(inf['path_train'], topic_number=inf['topic_number'], num_collection_passes=inf['num_collection_passes'])

inf['theta_test'] = wwm.get_theta_from_vw(inf, inf['path_test'])
inf['theta_castom'] = wwm.get_theta_from_vw(inf, inf['path_castom'])

/Users/kozlinskii/Documents/study/mipt/nir/service_cards/data_preparing.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  docs['text'] = docs['name'].map(lambda x: x if pd.notnull(x) else "") + " " + docs['description'].map(
/Users/kozlinskii/Documents/study/mipt/nir/service_cards/data_preparing.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  docs['first'] = docs['parent_id'].map(lambda x: x.split('/')[1] if pd.notnull(x) else None)
/Users/kozlinskii/Documents/study/mipt/nir/service_cards/data_prepa

In [13]:
wwm.make_docs_dicts(inf)
wwm.form_card2topic(inf)
wwm.get_topic2rubric(inf)
wwm.measure_accuracy_on_test(inf)
print(inf.keys())

16750  /  66104 0.25338859978216144
dict_keys(['exp_name', 'test_size', 'num_collection_passes', 'topic_number', 'data_path', 'docs', 'path_train', 'path_test', 'path_castom', 'model', 'theta_train', 'theta_test', 'theta_castom', 'card2rubric', 'card2name', 'card2text', 'card2topic_castom', 'card2topic_train', 'card2topic_test', 'card2topic', 'topic2rubric'])


In [14]:
wwm.get_answers_on_castom_advanced(inf)

1  
prediction: 	 remont-i-stroitel_stvo/elektromontajnye-raboty/zamena-knopok-i-vyklucatelej 
 настоящий: 	  remont-i-stroitel_stvo/elektromontajnye-raboty/ 
 name:  замена электрощитка 
 замена электрощитка  


2  
prediction: 	 remont-i-stroitel_stvo/okna-i-balkony/montaj-okon 
 настоящий: 	  remont-i-stroitel_stvo/drugoe/ 
 name:  монтаж вентиляции и кондиционирования 
 монтаж вентиляции и кондиционирования проект консультации подбор оборудования монтаж 


3  
prediction: 	 remont-i-stroitel_stvo/zabory-i-ograjdenia/stroitel_stvo-zabora-iz-polikarbonata 
 настоящий: 	  remont-i-stroitel_stvo/oboi-i-malarnye-raboty/ 
 name:  оклейка обоев 
 оклейка обоев оклейка обоев работаю с 2009 года так же все подготовительные работы есть плотник плиточник его услуги по надобности есть множество фото и видио работ качество гарантирую 


4  
prediction: 	 remont-i-stroitel_stvo/vysotnye-raboty/okraska-fasadov 
 настоящий: 	  remont-i-stroitel_stvo/svarocnye-raboty/ 
 name:  иготовление двухскатн

In [15]:
def get_theta_from_vw_third(inf, path):
    batch_vectorizer = None
    batch_vectorizer = artm.BatchVectorizer(data_path='./' + path,
                                                  data_format='vowpal_wabbit',
                                                  target_folder='folder' + path)
    return inf['model'].transform(batch_vectorizer=batch_vectorizer, predict_class_id='third')

In [18]:
theta_castom_third = get_theta_from_vw_third(inf, inf['path_castom'])

In [19]:
theta_castom_third.shape

(0, 123865)

In [20]:
inf['theta_castom'].shape

(750, 123865)

In [23]:
theta_castom_third.T.head()

""
116d78d3-8f8a-4a55-b6ba-7c80874affb7
47118622-c96b-4b90-9151-581561fe308d
aaf45581-c4dd-4c0c-9732-af193529334b
9421b7e9-0676-43e0-bad8-bd0cb635cf23
0de64bef-bf29-4a50-ba62-adcd59afec41
